In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import os

In [3]:
df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")
df

C:\Users\lyoshka\AppData\Local\Temp\ipykernel_6468\203287006.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.621790,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220329,04/20/2021,7:00,NaN,NaN,NaN,NaN,NaN,LONG ISLAND EXPRESSWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4408866,Pick-up Truck,Tractor Truck Diesel,NaN,NaN,NaN
2220330,04/20/2021,15:07,NaN,NaN,40.683480,-73.966896,"(40.68348, -73.966896)",FULTON STREET,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4408962,Convertible,Moped,NaN,NaN,NaN
2220331,04/20/2021,17:00,BRONX,10458.0,40.855600,-73.885150,"(40.8556, -73.88515)",NaN,NaN,660 EAST 188 STREET,...,Unspecified,NaN,NaN,NaN,4409125,Sedan,NaN,NaN,NaN,NaN
2220332,04/20/2021,16:45,BROOKLYN,11204.0,40.625828,-73.990950,"(40.625828, -73.99095)",NaN,NaN,5822 16 AVENUE,...,NaN,NaN,NaN,NaN,4408955,Sedan,NaN,NaN,NaN,NaN


# Lidando com Missing Values

In [4]:
df.isnull().sum().sum() # Muitos valores nulos impossivel exclui-los

np.int64(18984049)

## Removendo Colunas com dados nulos acima de 82%

In [5]:
df.isna().sum() # colunas off street name, contributing factor 3 veicle ate o 5 , vehicle type code 3 ate o 5 sao colunas com muitos valores nulos e podem ser removidas
missing_percent = df.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))
df_removed = df.drop(columns=['OFF STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])
df_removed.isna().sum().sum()

VEHICLE TYPE CODE 5              99.561778
CONTRIBUTING FACTOR VEHICLE 5    99.547546
VEHICLE TYPE CODE 4              98.407987
CONTRIBUTING FACTOR VEHICLE 4    98.348041
VEHICLE TYPE CODE 3              93.045145
CONTRIBUTING FACTOR VEHICLE 3    92.764197
OFF STREET NAME                  82.379228
CROSS STREET NAME                38.211774
ZIP CODE                         30.650839
BOROUGH                          30.638408
ON STREET NAME                   21.781137
VEHICLE TYPE CODE 2              20.060270
CONTRIBUTING FACTOR VEHICLE 2    16.053486
LOCATION                         10.825353
LATITUDE                         10.825353
LONGITUDE                        10.825353
VEHICLE TYPE CODE 1               0.727458
CONTRIBUTING FACTOR VEHICLE 1     0.353145
NUMBER OF PERSONS KILLED          0.001396
NUMBER OF PERSONS INJURED         0.000811
CRASH DATE                        0.000000
NUMBER OF PEDESTRIANS INJURED     0.000000
CRASH TIME                        0.000000
NUMBER OF M

np.int64(4239834)

In [ ]:
# comparando antes e depois de remover os nulos
print(df.shape)
print(df_removed.shape)

(2220334, 29)
(2220334, 22)


In [ ]:
# demonstração de valores faltantes em ordem descendente
df_removed.isna().sum() 
missing_percent = df_removed.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))

CROSS STREET NAME                38.211774
ZIP CODE                         30.650839
BOROUGH                          30.638408
ON STREET NAME                   21.781137
VEHICLE TYPE CODE 2              20.060270
CONTRIBUTING FACTOR VEHICLE 2    16.053486
LOCATION                         10.825353
LATITUDE                         10.825353
LONGITUDE                        10.825353
VEHICLE TYPE CODE 1               0.727458
CONTRIBUTING FACTOR VEHICLE 1     0.353145
NUMBER OF PERSONS KILLED          0.001396
NUMBER OF PERSONS INJURED         0.000811
CRASH TIME                        0.000000
CRASH DATE                        0.000000
NUMBER OF CYCLIST INJURED         0.000000
NUMBER OF PEDESTRIANS INJURED     0.000000
NUMBER OF PEDESTRIANS KILLED      0.000000
NUMBER OF MOTORIST KILLED         0.000000
NUMBER OF MOTORIST INJURED        0.000000
NUMBER OF CYCLIST KILLED          0.000000
COLLISION_ID                      0.000000
dtype: float64


# Lidando Com Outliers

In [22]:
# Converte os outliars de longitude e latitude em Nan
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.26, -73.69
filtro_invalidas = (
    (df_removed['LATITUDE'] < lat_min) | (df_removed['LATITUDE'] > lat_max) |
    (df_removed['LONGITUDE'] < lon_min) | (df_removed['LONGITUDE'] > lon_max)
)
df.loc[filtro_invalidas, ['LATITUDE', 'LONGITUDE']] = np.nan

## Usando novo dataset sem colunas com muitos nulos para imputar restante dos resultados com simpleImputer 

In [23]:
num_cols_auto = df_removed.select_dtypes(include=['number']).columns.to_list()
cat_cols_auto = df_removed.select_dtypes(include=['object']).columns.to_list()
num_cols_imputar = [
    col for col in num_cols_auto 
    if col not in ['LATITUDE', 'LONGITUDE', 'COLLISION_ID'] 
]
cat_cols_imputar = cat_cols_auto


In [ ]:
transformers_imput = [
    ('num_imputer', SimpleImputer(strategy='median'), num_cols_imputar),
    ('freq_imputer', SimpleImputer(strategy='most_frequent'), cat_cols_imputar)
]
ct_imput = ColumnTransformer(transformers_imput, remainder='passthrough')
df_imputed_array = ct_imput.fit_transform(df_removed)

# Obter nomes das colunas 'remainder'
imputed_cols = num_cols_imputar + cat_cols_imputar
all_cols_original = df_removed.columns.tolist()
remainder_cols = [col for col in all_cols_original if col not in imputed_cols]

# Ordem correta das colunas no array
nova_ordem_colunas = num_cols_imputar + cat_cols_imputar + remainder_cols

# Reconstruir
df_imputed = pd.DataFrame(df_imputed_array, columns=nova_ordem_colunas)

colunas_numericas_finais = num_cols_imputar + remainder_cols

for col in colunas_numericas_finais:
    # pd.to_numeric é a melhor forma de fazer isso
    df_imputed[col] = pd.to_numeric(df_imputed[col], errors='coerce')

# 2. (Opcional) Garante que as colunas categóricas sejam 'string'
for col in cat_cols_imputar:
    df_imputed[col] = df_imputed[col].astype(str)

print("\n--- DEPOIS da Conversão de Tipo (Tipos Corrigidos) ---")
df_imputed.info()

In [25]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220334 entries, 0 to 2220333
Data columns (total 22 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   NUMBER OF PERSONS INJURED      object
 1   NUMBER OF PERSONS KILLED       object
 2   NUMBER OF PEDESTRIANS INJURED  object
 3   NUMBER OF PEDESTRIANS KILLED   object
 4   NUMBER OF CYCLIST INJURED      object
 5   NUMBER OF CYCLIST KILLED       object
 6   NUMBER OF MOTORIST INJURED     object
 7   NUMBER OF MOTORIST KILLED      object
 8   CRASH DATE                     object
 9   CRASH TIME                     object
 10  BOROUGH                        object
 11  ZIP CODE                       object
 12  LOCATION                       object
 13  ON STREET NAME                 object
 14  CROSS STREET NAME              object
 15  CONTRIBUTING FACTOR VEHICLE 1  object
 16  CONTRIBUTING FACTOR VEHICLE 2  object
 17  VEHICLE TYPE CODE 1            object
 18  VEHICLE TYPE CODE 2   

In [12]:
df_imputed.isna().sum() 
missing_percent = df_imputed.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))

LATITUDE                         0.0
LONGITUDE                        0.0
NUMBER OF PERSONS INJURED        0.0
NUMBER OF PERSONS KILLED         0.0
NUMBER OF PEDESTRIANS INJURED    0.0
NUMBER OF PEDESTRIANS KILLED     0.0
NUMBER OF CYCLIST INJURED        0.0
NUMBER OF CYCLIST KILLED         0.0
NUMBER OF MOTORIST INJURED       0.0
NUMBER OF MOTORIST KILLED        0.0
COLLISION_ID                     0.0
CRASH DATE                       0.0
CRASH TIME                       0.0
BOROUGH                          0.0
ZIP CODE                         0.0
LOCATION                         0.0
ON STREET NAME                   0.0
CROSS STREET NAME                0.0
CONTRIBUTING FACTOR VEHICLE 1    0.0
CONTRIBUTING FACTOR VEHICLE 2    0.0
VEHICLE TYPE CODE 1              0.0
VEHICLE TYPE CODE 2              0.0
dtype: float64


# Resolvendo inconsistências

In [17]:
try:
    # df = pd.read_csv('df_8_14_colunas.csv')
    # print(f"--- Carregado 'df_8_14_colunas.csv' para Identificação ---")
    
    # 1. Identificar todas as colunas que são do tipo 'object' (texto)
    object_cols = df_imputed.select_dtypes(include=['object']).columns
    
    if len(object_cols) == 0:
        print("Nenhuma coluna de texto (object) encontrada neste arquivo.")
    else:
        print(f"Colunas de texto (object) encontradas: {list(object_cols)}")
        
        # Limite para decidir se imprimimos todos os valores únicos
        low_cardinality_threshold = 50 
        
        print("\n--- 2. Análise de Valores Únicos (Identificação) ---")
        
        for col in object_cols:
            num_unique = df[col].nunique()
            
            print(f"\nColuna: '{col}' | Total de Valores Únicos: {num_unique}")
            
            # Se a coluna tiver poucos valores únicos, é categórica.
            # Vamos imprimir as contagens para o usuário ver.
            if num_unique <= low_cardinality_threshold:
                print("Contagem de valores (value_counts):")
                # Imprimir .value_counts() que já vem ordenado
                print(df[col].value_counts(dropna=False)) # dropna=False para vermos os NaNs
            else:
                print(f"Cardinalidade muito alta. Exibindo 5 valores de exemplo:")
                print(df[col].sample(5, random_state=1).to_list())
except FileNotFoundError:
    print("Erro: O arquivo 'df_8_14_colunas.csv' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Colunas de texto (object) encontradas: ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']

--- 2. Análise de Valores Únicos (Identificação) ---

Coluna: 'CRASH DATE' | Total de Valores Únicos: 4882
Cardinalidade muito alta. Exibindo 5 valores de exemplo:
['07/11/2012', '03/09/2016', '04/16/2016', '06/15/2023', '06/27/2014']

Coluna: 'CRASH TIME' | Total de Valores Únicos: 1440
Cardinalidade muito alta. Exibindo 5 valores de exemplo:
['11:08', '18:00', '14:00', '15:20', '16:00']

Coluna: 'BOROUGH' | Total de Valores Únicos: 5
Contagem de valores (value_counts):
BOROUGH
BROOKLYN         1173839
QUEENS            412674
MANHATTAN         341294
BRONX             228048
STATEN ISLAND      64479
Name: count, dtype: int64
Ocorreu um erro inesperado: 'ZIP CODE'


In [21]:
df_reduzido = df_imputed.iloc[:, 14:16]
df_reduzido.to_csv("df_14_16_colunas.csv", index=False)